# ORÁCULO: Primitiva

http://www.lotoideas.com/primitiva-resultados-historicos-de-todos-los-sorteos/


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

import time
import operator

Using TensorFlow backend.


In [2]:
debug = False
def log(*params):
    if debug:
        for x in params:
            print(x)

In [3]:
#Cargar datos
datos = pd.read_csv('datos/Primitiva_2013_2018.csv')

try:
    datos = datos.drop(labels=['comp'], axis=1)
except:
    None #Ya lo eliminaste en el fichero?

log(datos.head(), "\n")
    
#Poner los datos mas recientes al final...
datos = datos.reindex(index=datos.index[::-1])

log(datos.head(), "\n")

max = datos.max(1).max()
min = datos.min(1).min()
log("MAX: ", max)#49
log("MIN: ", min)# 1
log("DIM: ", datos.shape)
features = max - min +1

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
#Muestra una grafica de la evolucion del error de training y validacion
def grafica(hist, offset0 = 0, offset1 = None):
    loss = hist.history['loss'][offset0:offset1]
    val_loss = hist.history['val_loss'][offset0:offset1]
    epochs = range(offset0 + 1, len(loss) + offset0 + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [5]:
#Funciones para mostrar tiempo transcurrido
def print_time(time_ini):
    sec = round(time.time() - time_ini)
    min = 0
    if(sec > 60):
        min = sec // 60
        sec = sec & 60
    print("TIME: ", min, "m  ", sec, "s")

def ini_time():
    print("Procesando... ", time.ctime())
    return time.time()

In [6]:
import winsound
def beep():
    duration = 500  # millisecond
    freq = 900  # Hz
    winsound.Beep(freq, duration)
    winsound.Beep(freq, duration)
    winsound.Beep(freq, 2*duration)

In [7]:
#Calcula el error incurrido en la predicción
import math

##---------------------------------------------------------------------
def calc_error_all(pred, real):
    error = []
    for i in range(len(real)):
        error.append(0)
        for j in range(len(real[i])):
            if(real[i][j] not in pred[i]):
                error[i] += 1
    for i in range(len(error)):
        error[i] = math.floor(error[i]*100.0/6.0)
    return error

##---------------------------------------------------------------------
def calc_error_one(pred, real):
    error = 0
    for j in range(len(real)):
        if(real[j] not in pred):
            error += 100.0/6.0
    return error

##---------------------------------------------------------------------
def accuracy_all(model, x, y):
    y_pred = translate_all(model.predict(x))
    error = calc_error_all(y_pred, devectorize_all(y))
    err = 0
    for x in error:
        err += x
    error100 = err/len(error)
    acierto100 = 100 - error100
    return acierto100

def accuracy2_all(model, x, y):
    sum_error = 0
    for i in range(len(y)):
        print(".", end="")
        pred = model.predict(x[i:(i+1)+seq_len])
        pred = translate_one(pred[-1])
        real = devectorize_one(y[i])
        error  = calc_error_one(pred, real)
        sum_error += error
    print()
    error = sum_error/len(y)
    acierto = 100 - error
    return acierto

In [8]:
#TODO: Si el peor de los mejores esta repetido, alguno mejor podria no entrar en la lista
def translate_all(pred):
    numeros = []
    for i in range(pred.shape[0]):
        min_mas_alto = sorted(pred[i], key=float, reverse=True)[5]
        #print(i, " min: ", min_mas_alto)
        numeros.append([])
        for j in range(pred.shape[1]):
            if pred[i][j] >= min_mas_alto and len(numeros[i]) < 6:
                numeros[i].append(j)
        #print(i, " num: ", numeros[i])
    return numeros

def translate_one(pred):
    numeros = []
    min_mas_alto = sorted(pred, key=float, reverse=True)[5]
    for j in range(pred.shape[0]):
        if pred[j] >= min_mas_alto and len(numeros) < 6:
            numeros.append(j)
    return numeros

In [9]:
#Convierte los numeros enteros premiados en un array binario ONE HOT

##
def vectorize_all(sequences, dimension=features):
    res = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        res[i, sequence-min] = 1
    return res

##
def devectorize_one(valor):
    res = []
    for i in range(len(valor)):
        if(valor[i] > 0):
            res.append(i+min)
    return res
#print(devectorize(np.array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0])), "\n")

##
def devectorize_all(valores):
    i = 0
    res = []
    for valor in valores:
        res.append([])
        for j in range(len(valor)):
            if(valor[j] > 0):
                res[i].append(j+min)
        i+=1
    return res

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
numeros = datos.drop(labels=['fecha'], axis=1)

log("----------------------------------------\n")
log(numeros.head())
log("----------------------------------------\n")
log(np.array(numeros))
log("----------------------------------------\n")

num_hot = vectorize_all(np.array(numeros))

log(num_hot)
log("----------")
log(devectorize_one(num_hot[0]))
log(devectorize_one(num_hot[1]))
log(devectorize_one(num_hot[2]))
log("...")

In [11]:
#Divide los datos en secuencias que el modelo tendrá que aprender
seq_len = 100
secuencias = []
log(len(num_hot), seq_len)
for index in range(len(num_hot) - seq_len):
    secuencias.append(num_hot[index : index + seq_len])
secuencias = np.array(secuencias)
###
log(devectorize_one(num_hot[0]))
log(devectorize_one(secuencias[0][0]))
log()
log(devectorize_one(num_hot[seq_len-1]))
log(devectorize_one(secuencias[0][seq_len-1]))
log()
log(devectorize_one(num_hot[1]))
log(devectorize_one(secuencias[1][0]))
log()
log(devectorize_one(num_hot[1+seq_len-1]))
log(devectorize_one(secuencias[1][seq_len-1]))
log()

log(devectorize_one(secuencias[0][0]))
log("...")


In [12]:
#Divide los datos en Training y Test
log(secuencias.shape)

row = round(0.9 * secuencias.shape[0])
train = secuencias[:int(row), :]

log(train.shape)
log(devectorize_one(train[0][0]))
log(devectorize_one(secuencias[0][0]))

#np.random.shuffle(train)

x_train = train[:, :-1]#Todos menos el ultimo valor, que es el que hay que predecir
log(x_train.shape)
log(x_train[0][0])
y_train = train[:, -1] #El ultimo valor, que es el que interesa predecir
log(y_train.shape)

x_test = secuencias[int(row):, :-1]
log(x_test.shape)
y_test = secuencias[int(row):, -1]
log(y_test.shape)


In [13]:

log(x_train.shape,  seq_len)

log()
log(devectorize_one(num_hot[0]))
log(devectorize_one(x_train[0][0]))
log()
log(devectorize_one(num_hot[seq_len-2]))
log(devectorize_one(x_train[0][seq_len-2]))
log()
log(devectorize_one(num_hot[seq_len-1]))
log(devectorize_one(y_train[0]))
log()
log(devectorize_one(num_hot[x_train.shape[0]+seq_len-2]))
log(devectorize_one(x_test[0][seq_len-2]))
log()
log(devectorize_one(num_hot[x_train.shape[0]+seq_len-1]))


### -----------------------------------------------------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------------------------------------------------

### Objetivo minimo: 3 numeros de 6 => 50% !!!

model1 ->

4 capas LSTM, 1 densa, epoca=, batch=128, m0s/0e -> 

5 capas LSTM, 1 densa, epoca=150, batch=128, 4m20s/0e -> OK TRAIN = 12.94 / OK TEST = 14.98
5 capas LSTM, 1 densa, epoca=160, batch=128, 7m52s/0e -> OK TRAIN = 12.33 / OK TEST = 15.02

epoca > 160 -> peor
epoca < 150 -> peor

6 capas LSTM, 1 densa, epoca=150, batch=128, 5m52s/150e -> OK TRAIN = 12.90 / OK TEST = 11.07   (0.10842311382293701)

                       epoca=101                                      11.89             13.02


model2 ->

1 capa LSTM, 1 bidireccional, epoca=250, batch=128, 5m16s/250e -> OK TRAIN = 11.67 / OK TEST = 12.23


2 capas LSTM -> OK TEST = 13.565891472868188
2 y 3 capas bidireccionales -> OK TEST = 10

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
#Crea la RNN
#features = x_train.shape[2]
def build_model():
    model = models.Sequential()
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    #model.add(layers.Dropout(0.2))
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))

    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=False))
    #model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=features, activation='sigmoid'))
    #model.add(layers.Dense(units=features))
    #model.add(layers.Activation("linear"))

    #start = time.time()
    #model.compile(loss="mse", optimizer="rmsprop")
    model.compile(loss="mse", optimizer="rmsprop")
    #print("> Compilation Time : ", round(time.time() - start), " segundos")
    return model


In [ ]:
#Entrenar la RNN
batch_size=128
epochs = 165
model = build_model()
timeini = ini_time()
hist1 = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)
print_time(timeini)
beep()

Procesando...  Fri Feb  2 15:50:16 2018


In [ ]:
print("Val.Loss: ", hist1.history['val_loss'][epochs-1])
grafica(hist1)
grafica(hist1, 110)

In [ ]:
#Error
acierto100a = accuracy_all(model, x_train, y_train)
print("M1 ACIERTO TRAIN: %.2f -------------------------------" % acierto100a)
acierto100a = accuracy_all(model, x_test, y_test)
print("M1 ACIERTO TEST:  %.2f -------------------------------" % acierto100a)

In [94]:
if(acierto100a > 15):
    model.save("bumodels/lotoA_"+str(acierto100a)+".h5")

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Train again with the better epoch
timeini = ini_time()
best_epoch = 140
model2 = build_model()
hist2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=best_epoch, validation_split=0.05, verbose=0)
print_time(timeini)
beep()

In [ ]:
print("Val.Loss: ", hist2.history['val_loss'][best_epoch])
grafica(hist2, 10,25)

In [ ]:
#Error
acierto100b = accuracy_all(model2, x_train, y_train)
print("M2 ACIERTO TRAIN: %.2f -------------------------------" % acierto100b)
acierto100b = accuracy_all(model2, x_test, y_test)
print("M2 ACIERTO TEST:  %.2f -------------------------------" % acierto100b)

#print("ACIERTO TRAIN: %.2f -------------------------------" % accuracy2_all(model2, x_train, y_train))
#print("ACIERTO TEST:  %.2f -------------------------------" % accuracy2_all(model2, x_test, y_test))

numeros_ganadores = translate_all(model2.predict(x_test))
log("REAL: ", devectorize_one(y_test[0]), " vs PRED: ", numeros_ganadores[0], "\n")

In [28]:
if(acierto100b > 15):
    model2.save("bumodels/lotoB_"+str(acierto100b)+".h5")

In [ ]:
#TODO: Modificar porcentage TRAIN / TEST a ver como responde...

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
#Predecir
log(secuencias.shape)
log(secuencias[-seq_len-1:-1].shape)
numeros_ganadores = model.predict(secuencias[-seq_len:-1])
numeros_ganadores = translate_one(numeros_ganadores[-1])
print(numeros_ganadores, " vs real ", devectorize_one(secuencias[-1][-1]))
print()

numeros_ganadores = model.predict(secuencias[-seq_len:])
numeros_ganadores = translate_one(numeros_ganadores[-1])
print("----------------------------------------------------------------")
print(numeros_ganadores)
print("----------------------------------------------------------------")


(430, 100, 49)
(100, 100, 49)
[11, 29, 36, 42, 45, 48]  vs real  [3, 6, 25, 31, 32, 40]

----------------------------------------------------------------
[11, 29, 36, 42, 45, 48]
----------------------------------------------------------------


In [ ]:
print("X:\n========")
print(x_train.shape)
print()
n = 0
for i in x_train:
    m = 0
    if(n < 2 or n > x_train.shape[0]-2):
        for j in i:
            if(m < 2 or m > i.shape[0]-2):
                print(n, m, devectorize_one(j))
            elif(m==10):
                print("...")
            m += 1
        print("-----------------------------------")
    elif(n==10):
        print("[...]\n")
    n += 1

    
print("\nY:\n========")
print(y_train.shape)
print()
n = 0
for i in y_train:
    if(n < 2 or n > y_train.shape[0]-2):
        print(n,devectorize_one(i))
    elif(n==10):
        print("...")
    n += 1

In [ ]:
log(devectorize_one(x_train[1][-1]), " corresponde con ", devectorize_one(y_train[0]))
log(devectorize_one(x_train[2][-1]), " corresponde con ", devectorize_one(y_train[1]))

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [77]:
#RNN LSTM Bidireccional
def build_model2():
    model = models.Sequential()
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    
    #model.add(layers.Embedding(features, 32))
    model.add(layers.Bidirectional(layers.LSTM(input_shape=(None, features), units=features, return_sequences=True)))
    model.add(layers.Bidirectional(layers.LSTM(input_shape=(None, features), units=features, return_sequences=False)))
    
    model.add(layers.Dense(features, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model

In [78]:
#Entrenar la RNN
batch_size=128
epochs = 150
model3 = build_model2()
timeini = ini_time()
hist3 = model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)
print_time(timeini)
beep()

Procesando...  Fri Feb  2 15:02:14 2018
TIME:  7 m   36 s


In [ ]:
print("Val.Loss: ", hist3.history['val_loss'][epochs-1])
grafica(hist3)
grafica(hist3, 10, 40)

In [ ]:
#Error
acierto100c = accuracy_all(model2, x_train, y_train)
print("M3 ACIERTO TRAIN: %.2f -------------------------------" % acierto100c)
acierto100c = accuracy_all(model2, x_test, y_test)
print("M3 ACIERTO TEST:  %.2f -------------------------------" % acierto100c)
print()
#print("ACIERTO TRAIN: %.2f -------------------------------" % accuracy2_all(model3, x_train, y_train))
#print("ACIERTO TEST:  %.2f -------------------------------" % accuracy2_all(model3, x_test, y_test))

In [40]:
if(acierto100c > 15):
    model3.save("bumodels/lotoC_"+str(acierto100c)+".h5")